In [1]:
# Load packages and Files

from pyhipp.io import h5
import numpy as np
import argparse
import h5py
from pathlib import Path
import json

from pyhipp_sims.sims.sim_info import SimInfo
from scipy.fft import rfftn, irfftn

import os
import time
from multiprocessing import Pool, cpu_count
from functools import partial

print("Package loaded successfully.")

BASE_PATH = Path('/data14/Hao_L/TNG100-1-Dark')
OUTPUT_PATH = Path('/data14/Fhong/halo_web')
CONFIG_FILE = OUTPUT_PATH / 'tng100-1-dark.json'
PREPROCESSED_FILE = OUTPUT_PATH / 'tng100-1-dark_postprocessed.hdf5'

num_processes = 8

data = h5py.File(PREPROCESSED_FILE, "r", locking=False)
if data:
    print("Raw data file opened successfully.")

Package loaded successfully.
Raw data file opened successfully.


In [2]:
# Generate Halo Samples

MIN_LOG_MASS = 10.5
MAX_LOG_MASS = 11.0
EXCLUDE_BACKSPLASH = True

OUTPUT_FILE = OUTPUT_PATH / 'halo_sample.hdf5'

print("--- Selecting Final Halo Sample ---")

# --- Loading File ---
print(f"Loading simulation config from: {CONFIG_FILE}")
config_file_path = Path(CONFIG_FILE)
if not config_file_path.exists():
    print(f"FATAL ERROR: Configuration file not found at '{config_file_path}'")
    
with open(config_file_path, 'r') as f:
    config_data = json.load(f)

sim_info = SimInfo(
    name=config_data['name'],
    root_dir=Path(BASE_PATH),
    root_file=Path(config_data['root_file']),
    sim_def=config_data['simulation_definition']
)

input_file_path = Path(PREPROCESSED_FILE)
if not input_file_path.exists():
    print(f"FATAL ERROR: Pre-processed file not found at '{input_file_path}'")
    print("Please run Stage 1 script first.")
    
print(f"Loading pre-processed catalog from: {input_file_path}")
with h5py.File(input_file_path, 'r', locking=False) as f_in:
    props = {key: val[:] for key, val in f_in['properties'].items()}
    h = 0.6774

# --- Halo Selection ---
print("Applying final selection criteria...")

mass_Msun = props['m_mean200'] * 1e10 / h
log_mass = np.log10(mass_Msun, where=mass_Msun > 0)
mass_mask = (log_mass >= MIN_LOG_MASS) & (log_mass < MAX_LOG_MASS)

final_mask = mass_mask
if EXCLUDE_BACKSPLASH:
    backsplash_mask = props['last_sat_snap'] == -1
    final_mask &= backsplash_mask
    
final_indices = np.where(final_mask)[0]

if len(final_indices) == 0:
    print("\nError: No halos found matching all criteria.")
    print(f"   - Found {np.sum(mass_mask)} halos in the mass range.")
    if EXCLUDE_BACKSPLASH:
        print(f"   - Of those, {np.sum(mass_mask & backsplash_mask)} also passed the backsplash cut.")
    
print(f"Final selection: {len(final_indices)} halos match the paper's criteria.")

# --- snap_form -> zf ---
print("Converting 'snap_form' to 'zf' and preparing final dataset...")

snap_form = props['snap_form'][final_indices].astype(int)

valid_snaps_mask = (snap_form >= 0) & (snap_form < len(sim_info.redshifts))

zf = np.full_like(snap_form, -1.0, dtype=np.float32)

valid_snap_indices = snap_form[valid_snaps_mask]
zf[valid_snaps_mask] = sim_info.redshifts[valid_snap_indices]

final_data = {
    'mass': props['m_crit200'][final_indices] * 1e10 / h,
    'zf': zf,
    'position': props['x'][final_indices],
    'm_peak': props['m_peak'][final_indices] * 1e10 / h,
    'v_peak': props['v_peak'][final_indices],
    'spin': props['spin'][final_indices]
}

output_file = Path(OUTPUT_FILE)
output_file.parent.mkdir(parents=True, exist_ok=True)

with h5py.File(output_file, 'w', locking=False) as f_out:
    f_out.attrs['simulation'] = sim_info.formal_name or sim_info.name
    f_out.attrs['n_halos'] = len(final_indices)
    f_out.attrs['mass_range_logMsun'] = [MIN_LOG_MASS, MAX_LOG_MASS]
    f_out.attrs['backsplash_excluded'] = EXCLUDE_BACKSPLASH
    
    for key, data in final_data.items():
        f_out.create_dataset(key, data=data)
        
print(f"\nFinal halo sample for analysis saved to: {output_file}")
print("Script finished successfully!")

--- Selecting Final Halo Sample ---
Loading simulation config from: /data14/Fhong/halo_web/tng100-1-dark.json
Loading pre-processed catalog from: /data14/Fhong/halo_web/tng100-1-dark_postprocessed.hdf5
Applying final selection criteria...
Final selection: 23653 halos match the paper's criteria.
Converting 'snap_form' to 'zf' and preparing final dataset...

Final halo sample for analysis saved to: /data14/Fhong/halo_web/halo_sample.hdf5
Script finished successfully!


In [ ]:
# Generate Field Samples

snapshot_dir = BASE_PATH / 'output'
sim_info_file = OUTPUT_PATH / 'tng100-1-dark.json'
snapshot_num = 99

# T-Web Algorithm Parameters
grid_size = 256  # The number of grid cells along one dimension
smoothing_scale = 0.3  # Gaussian smoothing scale in Mpc/h, as used in theory.ipynb
lambda_th = 0.2  # Eigenvalue threshold for web classification, as used in theory.ipynb

# Output file
output_file = OUTPUT_PATH / f'field_sample_{grid_size}_hr.hdf5'

# --- UTILITY FUNCTIONS ---

def get_para(snap_dir: Path, snap_num: int) -> dict:
    first_file_path = snap_dir / f'snap_{snap_num:03d}.0.hdf5'
    print(f"Reading header from: {first_file_path}")
    with h5py.File(first_file_path, 'r', locking=False) as f:
        header = f['Header'].attrs
        box_size_ckpc_h = header['BoxSize']      # In ckpc/h 
        num_files = header['NumFilesPerSnapshot'] # Number of chunks 
        
    return {
        'box_size_mpc_h': box_size_ckpc_h / 1000.0,
        'num_files': num_files
    }

def cic_deposit_chunk(positions: np.ndarray, box_size: float, n_grid: int, target_field: np.ndarray):
    pos_grid = positions / box_size * n_grid
    
    i = np.floor(pos_grid).astype(int)
    f = pos_grid - i
    
    w = [(1 - f, f), (1 - f, f), (1 - f, f)]
    
    for dx in [0, 1]:
        for dy in [0, 1]:
            for dz in [0, 1]:
                weight = w[0][dx][:, 0] * w[1][dy][:, 1] * w[2][dz][:, 2]
                indices = ( (i[:, 0] + dx) % n_grid, 
                            (i[:, 1] + dy) % n_grid, 
                            (i[:, 2] + dz) % n_grid )
                np.add.at(target_field, indices, weight)

def process_chunk(chunk_index: int, snap_dir: Path, snap_num: int, box_size: float, n_grid: int) -> tuple[np.ndarray, int]:
    file_path = snap_dir / f'snap_{snap_num:03d}.{chunk_index}.hdf5'
    print(f"[Worker {os.getpid()}] Processing chunk {chunk_index}...")

    partial_density_field = np.zeros((n_grid, n_grid, n_grid), dtype=np.float32)
    num_particles_in_chunk = 0

    try:
        with h5py.File(file_path, 'r', locking = False) as f:
            if 'PartType1' in f and 'Coordinates' in f['PartType1']:
                positions_chunk = f['PartType1']['Coordinates'][:].astype(np.float32) / 1000.0
                num_particles_in_chunk = positions_chunk.shape[0]
                cic_deposit_chunk(positions_chunk, box_size, n_grid, partial_density_field)
    except FileNotFoundError:
        print(f"  [Worker {os.getpid()}] WARNING: File {file_path} not found. Skipping.")

    print(f"[Worker {os.getpid()}] Finished chunk {chunk_index}, found {num_particles_in_chunk} particles.")
    return partial_density_field, num_particles_in_chunk


snapshot_dir = BASE_PATH / f'output/snapdir_{snapshot_num:03d}'
    
sim_params = get_para(snapshot_dir, snapshot_num)
box_size = sim_params['box_size_mpc_h']
num_files = sim_params['num_files'] 

start_time = time.time()

task_func = partial(process_chunk, 
                        snap_dir=snapshot_dir, 
                        snap_num=snapshot_num, 
                        box_size=box_size, 
                        n_grid=grid_size)
    
chunk_indices = range(num_files)

print(f"\nStarting parallel density field calculation with {num_processes} processes...")
with Pool(processes=num_processes) as pool:
    results = pool.map(task_func, chunk_indices)


print("\nParallel processing finished. Aggregating results...")
partial_fields = [res[0] for res in results]
particle_counts = [res[1] for res in results]

density_field = np.sum(partial_fields, axis=0, dtype=np.float32)
total_particles = np.sum(particle_counts)
del partial_fields, results 

print(f"Total particles aggregated: {total_particles}")
time_after_cic = time.time()


print(f"--- Density field generation took {time_after_cic - start_time:.2f} seconds ---")
mean_density = total_particles / grid_size**3
delta_field = density_field / mean_density - 1.0
del density_field

print("Calculating tidal field via FFT...")
delta_k = rfftn(delta_field)

k_values = 2 * np.pi * np.fft.fftfreq(grid_size, d=box_size / grid_size)
kx, ky, kz_rfft = np.meshgrid(k_values, k_values, k_values[:grid_size//2 + 1], indexing='ij', sparse=True)

k_sq = kx**2 + ky**2 + kz_rfft**2
k_sq[0, 0, 0] = 1.0

smoothing_factor_k = np.exp(-0.5 * k_sq * smoothing_scale**2)
delta_sm_k = delta_k * smoothing_factor_k

T_ij_k = {}
T_ij_k['11'] = (kx * kx / k_sq - 1/3) * delta_sm_k
T_ij_k['12'] = (kx * ky / k_sq)       * delta_sm_k
T_ij_k['13'] = (kx * kz_rfft / k_sq)  * delta_sm_k
T_ij_k['22'] = (ky * ky / k_sq - 1/3) * delta_sm_k
T_ij_k['23'] = (ky * kz_rfft / k_sq)  * delta_sm_k
T_ij_k['33'] = (kz_rfft * kz_rfft / k_sq - 1/3) * delta_sm_k

print("Inverse transforming back to real space...")
T_ij = {key: irfftn(val, s=(grid_size,)*3) for key, val in T_ij_k.items()}
delta_sm_x = irfftn(delta_sm_k, s=(grid_size,)*3)

print("Calculating eigenvalues...")
tidal_tensor = np.array([
    [T_ij['11'], T_ij['12'], T_ij['13']],
    [T_ij['12'], T_ij['22'], T_ij['23']],
    [T_ij['13'], T_ij['23'], T_ij['33']]
])
tidal_tensor = np.moveaxis(tidal_tensor, [0, 1], [-2, -1])
eigenvalues = np.linalg.eigh(tidal_tensor)[0][..., ::-1]

print(f"Saving results to {output_file}...")
with h5py.File(output_file, 'w', locking = False) as f:
    f.create_dataset('lams', data=eigenvalues.astype(np.float32))
    f.create_dataset('delta_sm_x', data=delta_sm_x.astype(np.float32))
    f.create_dataset('l_box', data=box_size)
    f.create_dataset('n_grids', data=grid_size)
    n_above_thresh = (eigenvalues > lambda_th).sum(axis=3)
    f.create_dataset('web_type', data=n_above_thresh.astype(np.int8))
    f.attrs['smoothing_scale_mpc_h'] = smoothing_scale
    f.attrs['lambda_th'] = lambda_th
    
print("Done!")

Reading header from: /data14/Hao_L/TNG100-1-Dark/output/snapdir_099/snap_099.0.hdf5

Starting parallel density field calculation with 8 processes...
[Worker 20442] Processing chunk 4...[Worker 20443] Processing chunk 6...[Worker 20441] Processing chunk 2...[Worker 20444] Processing chunk 8...[Worker 20445] Processing chunk 10...[Worker 20446] Processing chunk 12...[Worker 20440] Processing chunk 0...


[Worker 20447] Processing chunk 14...




[Worker 20440] Finished chunk 0, found 94082254 particles.
[Worker 20440] Processing chunk 1...
[Worker 20447] Finished chunk 14, found 94099645 particles.
[Worker 20447] Processing chunk 15...
[Worker 20441] Finished chunk 2, found 94168133 particles.
[Worker 20441] Processing chunk 3...
[Worker 20442] Finished chunk 4, found 93802639 particles.
[Worker 20442] Processing chunk 5...
[Worker 20446] Finished chunk 12, found 94152614 particles.
[Worker 20443] Finished chunk 6, found 94467000 particles.
[Worker 20446] Processing chunk 13...
[Worker 2